In [4]:
import pandas as pd
import numpy as np
from datetime import timedelta
import matplotlib.pyplot as plt
from scipy.stats import linregress
from pandas.tseries.offsets import DateOffset
import os


In [15]:
# Step 1: Initial setup
def prepare_time_series(file_path, startyardage_value):
    df = pd.read_excel(file_path)
    df = df[df['STARTYARDAGE'] == startyardage_value].copy()

    df['MCAL_DAY_DT'] = pd.to_datetime(df['MCAL_DAY_DT'], errors='coerce')
    df = df.dropna(subset=['MCAL_DAY_DT'])
    df = df.sort_values('MCAL_DAY_DT')
    df = df.drop_duplicates(subset=['MCAL_DAY_DT'], keep='first')
    return df[['MCAL_DAY_DT', 'VAL_WT35']]

# Step 2: Detecting interventions
def detect_large_drops_as_interventions(df, value_col, drop_threshold=-.10):
    df['Pct_Change'] = df[value_col].pct_change()
    df['Suspected_Intervention'] = df['Pct_Change'] <= drop_threshold
    return df

df = detect_large_drops_as_interventions(df, 'VAL_WT35')

# Step 3: Segmentation of data based on interventions
## Update: to handle the repitation and thereafter the droppings of the duplicates I am resetting the index. 

def segment_data(df, intervention_col):
    df = df.reset_index(drop=True) # Reset the DataFrame index to ensure continuity
    regimes = []
    start_idx = 0  # Start from the first position of the DataFrame after resetting index
    for i, row in enumerate(df.itertuples(), 1):  # Enumerate starting from 1 for correct slicing
        if getattr(row, intervention_col):
            end_idx = i
            if start_idx != end_idx:  
                regimes.append(df.iloc[start_idx:end_idx-1])  
            start_idx = end_idx  
    if start_idx < len(df):
        regimes.append(df.iloc[start_idx:])
    
    return regimes

regimes = segment_data(df, 'Suspected_Intervention')

# The details of each regime
for i, regime in enumerate(regimes):
    if not regime.empty:
        print(f"Regime {i+1}: Start Date: {regime['MCAL_DAY_DT'].iloc[0]}, End Date: {regime['MCAL_DAY_DT'].iloc[-1]}, Length: {len(regime)}")
    else:
        print(f"Regime {i+1}: Empty Regime")


# Step 4: Calculate the slope for each regime
def calculate_regime_slopes(regimes, date_col, value_col):
    slopes = []
    for regime in regimes:
        if len(regime) > 1:
            days = (regime[date_col] - regime[date_col].min()).dt.days
            slope, _, _, _, _ = linregress(days, regime[value_col])
            slopes.append(slope)
        else:
            slopes.append(0)  # In case there's a regime with a single point, set slope to 0
    return slopes

slopes = calculate_regime_slopes(regimes, 'MCAL_DAY_DT', 'VAL_WT35')

# Step 5: Predict future values

def predict_future_values(df, date_col, value_col, regimes, slopes, num_days=730):
    last_regime = regimes[-1]
    
    # If the last regime is shorter than 12 data points, calculate a weighted average
    if len(last_regime) < 12 and len(slopes) >= 2:
        # Use the length of each regime as the weight
        weights = [len(regime) for regime in regimes[-2:]]  # Getting the length of the last two regimes
        weighted_slope = sum(w * s for w, s in zip(weights, slopes[-2:])) / sum(weights)
    else:
        weighted_slope = slopes[-1]  # Use the last slope directly if there's only one regime or it's long enough

    # If slope is negative set it to zero
    weighted_slope = max(weighted_slope, 0)

    last_date = df[date_col].max()
    last_value = df.loc[df[date_col] == last_date, value_col].values[0]

    future_dates = [last_date + DateOffset(days=i) for i in range(1, num_days + 1)]
    future_values = [last_value + weighted_slope * i for i in range(1, num_days + 1)]

    return future_dates, future_values



# Step 6: Plot the results
def plot_results(df, date_col, value_col, intervention_col, future_dates, future_values, output_dir, startyardage_value, regimes):
    plt.figure(figsize=(14, 7))

    # Original data
    plt.plot(df[date_col], df[value_col], label='Original')

    # Interventions
    interventions = df[df[intervention_col]]
    plt.scatter(interventions[date_col], interventions[value_col], color='red', label='Suspected Intervention', zorder=5)

    # Future predictions
    plt.plot(future_dates, future_values, 'b--', label='Forecasted')
    plt.ylim(0, 10)  

    # Danger lines
    plt.axhline(y=5.3, color='blue', linestyle='-', linewidth=0.5)
    plt.axhline(y=4, color='red', linestyle='-', linewidth=0.5)
    plt.axhline(y=3.2, color='yellow', linestyle='-', linewidth=0.5)
    plt.axhline(y=2.2, color='green', linestyle='-', linewidth=0.5)


    # Highlight points used for prediction (Last and Second Last Regime) while ensuring there are at least two regimes to plot both markers
    if len(regimes) >= 2:
        # Highlight the Second Last Regime
        second_last_regime = regimes[-2]
        plt.scatter(second_last_regime[date_col], second_last_regime[value_col], marker='v', color='purple', label='Second Last Regime', zorder=6)
        
    # Highlight the Last Regime
    last_regime = regimes[-1]
    plt.scatter(last_regime[date_col], last_regime[value_col], marker="s", color='green', label='Last Regime', zorder=6)

    plt.xlabel('Date')
    plt.ylabel(value_col)
    plt.title(f'Time Series Analysis with Intervention Detection and Forecasting for STARTYARDAGE {startyardage_value}')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()

    # Save the figure
    filename = f"Time_Series_{startyardage_value}.png"
    plt.savefig(os.path.join(output_dir, filename))
    plt.close() 

# Step 6: Plot the results
def plot_results(df, date_col, value_col, intervention_col, future_dates, future_values, output_dir, startyardage_value, regimes):
    plt.figure(figsize=(14, 7))

    # Original data
    plt.plot(df[date_col], df[value_col], label='Original')

    # Interventions
    interventions = df[df[intervention_col]]
    plt.scatter(interventions[date_col], interventions[value_col], color='red', label='Suspected Intervention', zorder=5)

    # Future predictions
    if len(set(future_values)) == 1:
        future_line_style = 'r--'  # Red dashed line for slope of 0
        future_label = 'Forecasted (Flat)'
    else:
        future_line_style = 'b--'  
        future_label = 'Forecasted'

    plt.plot(future_dates, future_values, future_line_style, label=future_label)
    plt.ylim(0, 10)  

    # Danger lines
    plt.axhline(y=5.3, color='blue', linestyle='-', linewidth=0.5)
    plt.axhline(y=4, color='red', linestyle='-', linewidth=0.5)
    plt.axhline(y=3.2, color='yellow', linestyle='-', linewidth=0.5)
    plt.axhline(y=2.2, color='green', linestyle='-', linewidth=0.5)

    # Highlight points used for prediction (Last and Second Last Regime) while ensuring there are at least two regimes to plot both markers
    if len(regimes) >= 2:
        second_last_regime = regimes[-2]
        plt.scatter(second_last_regime[date_col], second_last_regime[value_col], marker='v', color='purple', label='Second Last Regime', zorder=6)
        
    last_regime = regimes[-1]
    plt.scatter(last_regime[date_col], last_regime[value_col], marker="s", color='green', label='Last Regime', zorder=6)

    # Usual stuff
    plt.xlabel('Date')
    plt.ylabel(value_col)
    plt.title(f'Time Series Analysis with Intervention Detection and Forecasting for STARTYARDAGE {startyardage_value}')
    plt.legend()
    plt.grid(True)
    plt.xticks(rotation=45)
    plt.tight_layout()
    filename = f"Time_Series_{startyardage_value}.png"
    plt.savefig(os.path.join(output_dir, filename))
    plt.close()

    
def process_and_plot_for_all_startyardage(file_path, output_dir):
    # Read the Excel file once to get all unique STARTYARDAGE values
    df = pd.read_excel(file_path)
    unique_startyardage_values = df['STARTYARDAGE'].unique()

    # Ensure the output directory exists
    if not os.path.exists(output_dir):
        os.makedirs(output_dir)

    # Loop through all unique STARTYARDAGE values
    for startyardage_value in unique_startyardage_values:
        df_filtered = prepare_time_series(file_path, startyardage_value)
        df_with_interventions = detect_large_drops_as_interventions(df_filtered, 'VAL_WT35')
        regimes = segment_data(df_with_interventions, 'Suspected_Intervention')
        slopes = calculate_regime_slopes(regimes, 'MCAL_DAY_DT', 'VAL_WT35')
        future_dates, future_vals = predict_future_values(df_with_interventions, 'MCAL_DAY_DT', 'VAL_WT35', regimes, slopes, num_days=730)

        plot_results(df_with_interventions, 'MCAL_DAY_DT', 'VAL_WT35', 'Suspected_Intervention', future_dates, future_vals, output_dir, startyardage_value, regimes)


file_path = r'C:\Users\AMahmud1\Downloads\TQdata\Book_Meta.xlsx'
output_dir = r'C:\Users\AMahmud1\Downloads\TQdata\Plots_WG_NonNeg'  
process_and_plot_for_all_startyardage(file_path, output_dir)


Regime 1: Start Date: 2021-02-02 00:00:00, End Date: 2022-01-05 00:00:00, Length: 4
Regime 2: Start Date: 2022-03-01 00:00:00, End Date: 2023-01-01 00:00:00, Length: 8
Regime 3: Start Date: 2023-05-03 00:00:00, End Date: 2023-07-08 00:00:00, Length: 2


KeyboardInterrupt: 